# Committee Spending Per Year

*Author: Alan Leggitt (leggitta3@gmail.com)*  
*Date: 2017-06-11*

Visualizes the annual spending of the top spending committees.

In [1]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# connect to the database
engine = create_engine(
    "mysql://reader@localhost/campaign_expenditures")

# connect bokeh
output_notebook()

Loading BokehJS ...

In [3]:
# identify the top spenders
committee_query = """
    SELECT
        fec_committee_id,
        SUM(expenditures.amount) as total_spent
    FROM
        filings
    INNER JOIN
        expenditures
    ON
        expenditures.filing_id = filings.filing_id
    GROUP BY
        fec_committee_id
    ORDER BY
        total_spent DESC
    LIMIT 5;
"""
committee_data = pd.read_sql_query(committee_query, engine)

In [4]:
# open a new figure
fig = figure(width=800, height=800, background_fill_color='oldlace')
colors = ('red', 'blue', 'cyan', 'green', 'magenta')
fig.xaxis.axis_label = "Year"
fig.yaxis.axis_label = "Total Spending ($)"

# loop through the committees
for i, committee_id in enumerate(committee_data.fec_committee_id.values):

    # define query
    spending_query = """
        SELECT
            YEAR(expenditures.date) as year, SUM(expenditures.amount) as total_spent
        FROM
            expenditures
        INNER JOIN
            filings
        ON
            expenditures.filing_id = filings.filing_id
        WHERE
            filings.fec_committee_id = '%s'
            AND YEAR(expenditures.date) > 2000
            AND YEAR(expenditures.date) < 2018
        GROUP BY
            year
        ORDER BY
            year
    ;""" % committee_id
    
    # execute query and return data frame
    yearly_spending = pd.read_sql_query(spending_query, engine)
    
    # query the most recent name (as indicated by filing order)
    name_query = """
        SELECT
            committee_name
        FROM
            filings
        WHERE
            fec_committee_id = '%s'
        ORDER BY
            filing_id DESC
        LIMIT 1
    ;""" % committee_id
    cname = pd.read_sql_query(name_query, engine)
    cname = cname.committee_name.values[0]

    x = yearly_spending.year.values
    y = yearly_spending.total_spent.values

    fig.line(x, y, color=colors[i], legend=cname)
    fig.circle(x, y, color=colors[i], legend=cname)

fig.legend.location = "top_center"
show(fig)